In [1]:
import pandas as pd
import numpy as np

In [4]:
dfShootings = pd.read_csv('datasets/shootings.csv')
usPop = pd.read_csv('datasets/acs2017_census_tract_data.csv')

In [7]:
usPop['Other'] = round(100 - usPop['Hispanic'] - usPop['White'] - usPop['Black'] - usPop['Native'] - usPop['Asian'], 2)
orderedRace = ['Asian','Black','Hispanic','Native','Other','White']

usRacePop = pd.DataFrame(columns = [])
arrayRacePop = []
mortalityPerMillion = []
usTotalPop = usPop['TotalPop'].sum()

for item in orderedRace:
    usRacePop[item + 'Pop'] = round(usPop[item] * usPop['TotalPop'] / 100)
    arrayRacePop.append(round(usRacePop[item+'Pop'].sum() / usTotalPop, 4))
    
arrayRacePop

[0.0524, 0.1216, 0.1847, 0.0065, 0.0268, 0.6081]

In [ ]:
def sample_proportion(pop_size, prop, n=10000):
    grupo = pop_size * prop
    resultados = np.zeros(n)
    for i in range(n):
        sample = np.random.randint(0, pop_size, 100)
        resultados[i] = np.sum(sample < grupo)
    return resultados